# Day 24, Part 2

This is my third go at this problem.  In my first try, I missed some algebra tricks and didn't see how to turn it into a system of linear equations, so I treated it as an [optimization problem](https://github.com/bwbeach/advent-of-code/blob/main/2023/day24/part2.ipynb).  After reading Reddit, and seeing how to do the algebra, my [second try in Haskell](https://github.com/bwbeach/advent-of-code/tree/main/2023/day24b) worked fine, but I was underwhelmed by the linear algebra libraries.  This time, I'm going to take the same approach, but do it in Python and ~numpy~ `sympy`.

As with the second try in Haskell, we'll use rational numbers.  In Python, their called `Fraction`.

I tried a first pass at this with `numpy`, and it turns out that it doesn't support solving linear equations with numbers that are Fractions.  Several pages I found online recommended using `sympy` instead, so let's switch to using that.  And while we're at it, we can try solving the system of equations directly.

In [31]:
import fileinput
from fractions import Fraction
import sympy as sp
from sympy.abc import x, y, z, a, b, c  # will use a/b/c as the x/y/z components of rock velocity

In [30]:
# Read in the data into a list of hailstones.  
# Each is a pair of numpy arrays: (position, velocity)
#
# Input lines look like:
#    286311847445390, 152489168079800, 315302094634181 @ 29, -35, -385

def parse_line(s):
    """Parse one line and return a hailstone"""
    words = s.replace("@", "").replace(",", "").split()
    nums = [Fraction(w) for w in words]
    return (nums[:3], nums[3:])

hails = [parse_line(s) for s in fileinput.input("input.txt")]
hails[0]

([Fraction(197869613734967, 1),
  Fraction(292946034245705, 1),
  Fraction(309220804687650, 1)],
 [Fraction(150, 1), Fraction(5, 1), Fraction(-8, 1)])

In [33]:
# Functions to flatten hail from 3D to 2D

def dropX(hail):
    (x, y, z), (dx, dy, dz) = hail
    return ([y, z], [dy, dz])

def dropZ(hail):
    (x, y, z), (dx, dy, dz) = hail
    return ([x, y], [dx, dy])

# Make the two perspectives we'll be using
hails_xy = [dropZ(h) for h in hails]
hails_yz = [dropX(h) for h in hails]

hails_xy[:2]

[([Fraction(197869613734967, 1), Fraction(292946034245705, 1)],
  [Fraction(150, 1), Fraction(5, 1)]),
 ([Fraction(344503265587754, 1), Fraction(394181872935272, 1)],
  [Fraction(-69, 1), Fraction(11, 1)])]

In [35]:
def make_equation(h1, h2, x, y, a, b):
    """Makes an equation from two two-dimensional hailstones"""
    (pix, piy), (vix, viy) = h1
    (pkx, pky), (vkx, vky) = h2
    return (vky - viy) * x + (vix - vkx) * y + (piy - pky) * a + (pkx - pix) * b - (pkx * vky - pky * vkx - pix * viy + piy * vix)

make_equation(hails_xy[0], hails_xy[1], x, y, a, b)

-101235838689567*a + 146633651852787*b + 6*x + 219*y - 73940642222179977

In [44]:
def solve_2d(hails_2d, x, y, a, b):
    equations = [
        make_equation(hails_2d[i*2], hails_2d[i*2+1], x, y, a, b)
        for i in range(4)
    ]
    return sp.solve(equations, [x, y, a, b])

s1 = solve_2d(hails_xy, x, y, a, b)
s2 = solve_2d(hails_yz, x, z, a, c)

s1[x] + s1[y] + s2[z]

646810057104753